In [1]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning) 
import os
import joblib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging
import joblib
import json

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer
from numpy.linalg import LinAlgError
from scipy.stats import skew, kurtosis

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    RBF,
    Matern,
    RationalQuadratic,
    ExpSineSquared,
    DotProduct,
    WhiteKernel,
    ConstantKernel
)


In [2]:
def compute_volume_weighted_component_features(X):
    """
    Computes individual volume-weighted features WjPk = Componentj_fraction * Componentj_Propertyk
    for j in 1..5 and k in 1..10 (total 50 features).
    """
    features = {}
    for comp_idx in range(1, 6):  # Components 1–5
        for prop_idx in range(1, 11):  # Properties 1–10
            vol_col = f'Component{comp_idx}_fraction'
            prop_col = f'Component{comp_idx}_Property{prop_idx}'
            feat_name = f'W{comp_idx}P{prop_idx}'
            features[feat_name] = X[vol_col] * X[prop_col]
    return pd.DataFrame(features)

In [3]:
from sklearn.preprocessing import StandardScaler, PowerTransformer
from scipy.stats import skew
import numpy as np
import os
import pandas as pd

def get_data(target, skew_thresh=0.5):
    # Load train and val sets
    X_train = pd.read_csv(f"{BASE_PATH}/train/{target}_X.csv")
    y_train = pd.read_csv(f"{BASE_PATH}/train/{target}_y.csv")
    X_val = pd.read_csv(f"{BASE_PATH}/val/{target}_X.csv")
    y_val = pd.read_csv(f"{BASE_PATH}/val/{target}_y.csv")

    # Feature engineering
    X_train = pd.concat([X_train, compute_volume_weighted_component_features(X_train)], axis=1)
    X_val = pd.concat([X_val, compute_volume_weighted_component_features(X_val)], axis=1)

    # Feature selection
    df = pd.read_csv(os.path.join(fi_path, f"{target}.csv"))
    cols = df[df["importance"] > 0.1].iloc[:, 0].tolist()
    print(cols)
    X_train = X_train[cols]
    X_val = X_val[cols]

    # Separate out fraction-based columns
    fraction_cols = [col for col in X_train.columns if "fraction" in col.lower()]
    non_fraction_cols = [col for col in X_train.columns if col not in fraction_cols]

    # Initialize scalers
    x_scaler = StandardScaler()

    # Scale only non-fraction columns
    X_train_scaled = X_train.copy()
    X_val_scaled = X_val.copy()

    X_train_scaled[non_fraction_cols] = x_scaler.fit_transform(X_train[non_fraction_cols])
    X_val_scaled[non_fraction_cols] = x_scaler.transform(X_val[non_fraction_cols])
    # Fraction columns remain unchanged

    # y analysis
    y_vals = y_train.values.ravel()
    y_skew = skew(y_vals)
    should_transform_y = abs(y_skew) > skew_thresh

    if False:
        y_transformer = PowerTransformer(method="yeo-johnson")
        y_train_transformed = y_transformer.fit_transform(y_train.values.reshape(-1, 1)).ravel()
        y_val_transformed = y_transformer.transform(y_val.values.reshape(-1, 1)).ravel()
        print(f"⚠️ Applied PowerTransformer to target '{target}' (skewness: {y_skew:.2f})")
        return X_train_scaled, y_train_transformed, X_val_scaled, y_val_transformed, y_transformer
    else:
        print(f"✅ No transformation applied to target '{target}' (skewness: {y_skew:.2f})")
        return X_train_scaled, y_vals, X_val_scaled, y_val.values.ravel(), None


In [4]:
TARGETS = [f"BlendProperty{i}" for i in range(1, 11)]
BASE_PATH = "/pscratch/sd/r/ritesh11/temp_dir/dataset/updated"
model_dir = "/pscratch/sd/r/ritesh11/temp_dir/GPR_models"
fi_path = "/pscratch/sd/r/ritesh11/temp_dir/feature_importance"
N_TRIALS = 200

In [5]:
def get_kernel(trial):
    kernel_choice = trial.suggest_categorical("kernel", [
        "RBF", "Matern", "RQ", "DotProduct"
    ])

    scale = trial.suggest_float("const_scale", 0.1, 10.0)
    bias = trial.suggest_float("const_bias", 0.01, 10.0)

    if kernel_choice == "RBF":
        base_kernel = RBF(length_scale_bounds=(1e-5,1e5))

    elif kernel_choice == "Matern":
        nu = trial.suggest_categorical("matern_nu", [0.5, 1.5, 2.5])
        base_kernel = Matern(nu=nu,length_scale_bounds=(1e-5,1e5))

    elif kernel_choice == "RQ":
        base_kernel = RationalQuadratic(length_scale_bounds=(1e-5,1e5),alpha_bounds=(1e-5,1e5))

    # elif kernel_choice == "ExpSine":
    #     base_kernel = ExpSineSquared(length_scale_bounds=(1e-5,1e5),periodicity_bounds=(1e-5,1e5))

    elif kernel_choice == "DotProduct":
        base_kernel = DotProduct(sigma_0_bounds=(1e-5,1e5))

    # Compound kernel: scale * K() + bias + noise
    kernel = ConstantKernel(scale) * base_kernel
    kernel += ConstantKernel(bias)
    kernel += WhiteKernel(noise_level_bounds=(1e-5, 1e5))

    return kernel

In [6]:
fixed_params = {
    "random_state": 42,
    "optimizer" : "fmin_l_bfgs_b"
}

In [7]:
def objective(trial, X_train, y_train, X_val, y_val):
    kernel = get_kernel(trial) 

    params = {
        "kernel": kernel,
        "alpha": trial.suggest_float("alpha", 1e-12, 1e-3, log=True),
        "n_restarts_optimizer": trial.suggest_categorical("n_restarts_optimizer", [0, 1]),  
        "normalize_y": trial.suggest_categorical("normalize_y", [True, False]), 
    }
    try:
        model = GaussianProcessRegressor(**params,**fixed_params)
        model.fit(X_train, y_train)
    
        preds = model.predict(X_val)
        score = mean_absolute_percentage_error(y_val, preds)
        return score

    except LinAlgError as e:
        print(f"[Trial {trial.number}] LinAlgError: {e}")
        raise TrialPruned(f"Kernel matrix not PD: {e}")

    except ValueError as e:
        print(f"[Trial {trial.number}] ValueError: {e}")
        raise TrialPruned(f"Invalid kernel or GPR configuration: {e}")


In [8]:
def log_callback(study, trial):
        if trial.number % 50 == 0 and trial.value is not None:
            print(f"Trial {trial.number}: MAPE = {trial.value:.4f}: BEST: {study.best_trial.value:.4f}" )
        if trial.state.name == "PRUNED":
            print(f"⛔️ Trial {trial.number} was pruned. Reason: {trial.system_attrs.get('reason', 'unknown')}")

In [9]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
TARGETS = [TARGETS[2], TARGETS[4]]

In [10]:
for target in TARGETS:
     
    X_train, y_train, X_val, y_val, y_scaler = get_data(target)

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=N_TRIALS, callbacks=[log_callback],
                   n_jobs=12,show_progress_bar=True)

    print(f"\nBest MAPE for {target}: {study.best_value:.4f}")
    print(f"Best params for {target}:\n{study.best_params}\n")
    
    complete_params = {**study.best_params, **fixed_params}
    
    # Save best params (skip model training for now)
    os.makedirs(model_dir, exist_ok=True)
    with open(os.path.join(model_dir, f"best_params_{target}_updated_plain.json"), "w") as f:
        json.dump(complete_params, f, indent=2)

['Component5_fraction', 'Component4_fraction', 'Component2_fraction', 'Component1_fraction', 'W5P1', 'W1P1', 'W4P1', 'W3P1', 'W2P1', 'Component5_Property1', 'Component4_Property1', 'W1P9', 'Component2_Property1']
✅ No transformation applied to target 'BlendProperty1' (skewness: 0.05)


  0%|          | 0/200 [00:00<?, ?it/s]

Trial 0: MAPE = 0.0002: BEST: 0.0001
Trial 50: MAPE = 0.0001: BEST: 0.0001
Trial 100: MAPE = 0.0001: BEST: 0.0001
Trial 150: MAPE = 0.0001: BEST: 0.0001

Best MAPE for BlendProperty1: 0.0001
Best params for BlendProperty1:
{'kernel': 'DotProduct', 'const_scale': 4.98165609550217, 'const_bias': 4.666863808277834, 'alpha': 5.6700303337686935e-05, 'n_restarts_optimizer': 1, 'normalize_y': True}

['Component5_fraction', 'Component2_fraction', 'Component1_fraction', 'W5P2', 'Component4_fraction', 'W4P2', 'W3P2', 'W1P2', 'Component3_fraction', 'W2P2', 'Component4_Property2']
✅ No transformation applied to target 'BlendProperty2' (skewness: -0.04)


  0%|          | 0/200 [00:00<?, ?it/s]

Trial 0: MAPE = 0.0746: BEST: 0.0066
Trial 50: MAPE = 0.0067: BEST: 0.0065
Trial 100: MAPE = 0.0066: BEST: 0.0065
Trial 150: MAPE = 0.0066: BEST: 0.0065

Best MAPE for BlendProperty2: 0.0065
Best params for BlendProperty2:
{'kernel': 'RQ', 'const_scale': 5.146159919655902, 'const_bias': 6.935031069349357, 'alpha': 4.852163095642978e-10, 'n_restarts_optimizer': 1, 'normalize_y': True}

['Component2_fraction', 'Component4_fraction', 'Component5_fraction', 'W1P7', 'Component3_fraction', 'W3P7', 'W2P7', 'W2P8', 'W4P5', 'Component1_fraction', 'W4P7', 'W1P8', 'W3P5']
⚠️ Applied PowerTransformer to target 'BlendProperty3' (skewness: -0.57)


...

  0%|          | 0/200 [00:00<?, ?it/s]

Trial 0: MAPE = 0.3121: BEST: 0.3121
Trial 50: MAPE = 0.3121: BEST: 0.3119
Trial 100: MAPE = 0.3121: BEST: 0.3115
Trial 150: MAPE = 0.3122: BEST: 0.3115

Best MAPE for BlendProperty7: 0.2782
Best params for BlendProperty7:
{'kernel': 'Matern', 'const_scale': 3.543265383088894, 'const_bias': 4.9635450647153965, 'matern_nu': 1.5, 'alpha': 4.913055312535452e-05, 'n_restarts_optimizer': 1, 'normalize_y': False}

['Component5_fraction', 'Component2_fraction', 'W4P8', 'Component4_fraction', 'W3P8', 'W1P8', 'Component3_fraction', 'W2P9', 'W5P7', 'Component1_fraction', 'W2P8', 'W1P9', 'Component4_Property8', 'W4P7', 'W3P9', 'W5P5']
✅ No transformation applied to target 'BlendProperty8' (skewness: 0.09)


  0%|          | 0/200 [00:00<?, ?it/s]

Trial 0: MAPE = 0.2411: BEST: 0.2411
Trial 50: MAPE = 0.3189: BEST: 0.2071
Trial 100: MAPE = 0.2950: BEST: 0.2071
Trial 150: MAPE = 0.2411: BEST: 0.2061


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [21]:
# est MAPE for BlendProperty3: 0.5227
# Best MAPE for BlendProperty5: 0.6535

SyntaxError: invalid syntax (2279706546.py, line 1)